In [1]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

## 长上下文重排
无论您的模型的体系结构是什么，当您包含10个以上的检索文档时，都会有很大的性能下降。简而言之:当模型必须在长上下文中访问相关信息时，它们倾向于忽略所提供的文档。看到的:
[模型长上下文检索内容消失.pdf](http://185.242.235.221:8011/python/paper/%E9%95%BF%E4%B8%8A%E4%B8%8B%E6%96%87%E5%AF%BC%E8%87%B4%E6%96%87%E6%9C%AC%E6%A8%A1%E5%9E%8B%E4%B8%A2%E5%A4%B1%E4%BF%A1%E6%81%AF.pdf)
为了避免这个问题，您可以在检索后重新排序文档，以避免性能下降。


In [2]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

# Get embeddings.
embeddings = HuggingFaceEmbeddings(model_name=r"G:\model\all-MiniLM-L6-v2")

# 都是短文本
texts = [
    "Basquetball is a great sport.",
    "Fly me to the moon is one of my favourite songs.",
    "The Celtics are my favourite team.",
    "This is a document about the Boston Celtics",
    "I simply love going to the movies",
    "The Boston Celtics won the game by 20 points",
    "This is just a random text.",
    "Elden Ring is one of the best games in the last 15 years.",
    "L. Kornet is one of the best Celtics players.",
    "Larry Bird was an iconic NBA player.",
    "王小明喜欢上数学课",
    "张强决定进行一个新的项目",
    "将自己的技术把控在自己的身上",
    "以前的歌曲都听够了，现在最喜欢的歌是love story",
    "本年度最佳的互动奖颁发给了郭霖俊",
    "矮人科技适合于黎乐前辈",
    "科比打输了复活赛",
    "猫雷打赢了复活赛",
    "复活赛不是谁都能打赢的"
]

# Create a retriever
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)
query = "你能告诉我科比打没打赢复活赛?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='科比打输了复活赛'),
 Document(page_content='复活赛不是谁都能打赢的'),
 Document(page_content='猫雷打赢了复活赛'),
 Document(page_content='矮人科技适合于黎乐前辈'),
 Document(page_content='将自己的技术把控在自己的身上'),
 Document(page_content='张强决定进行一个新的项目'),
 Document(page_content='王小明喜欢上数学课'),
 Document(page_content='本年度最佳的互动奖颁发给了郭霖俊'),
 Document(page_content='以前的歌曲都听够了，现在最喜欢的歌是love story'),
 Document(page_content='This is just a random text.')]

In [3]:
# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at beginning / end.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(page_content='复活赛不是谁都能打赢的'),
 Document(page_content='矮人科技适合于黎乐前辈'),
 Document(page_content='张强决定进行一个新的项目'),
 Document(page_content='本年度最佳的互动奖颁发给了郭霖俊'),
 Document(page_content='This is just a random text.'),
 Document(page_content='以前的歌曲都听够了，现在最喜欢的歌是love story'),
 Document(page_content='王小明喜欢上数学课'),
 Document(page_content='将自己的技术把控在自己的身上'),
 Document(page_content='猫雷打赢了复活赛'),
 Document(page_content='科比打输了复活赛')]

In [4]:
# We prepare and run a custom Stuff chain with reordered docs as context.

# Override prompts
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
llm = OpenAI()
stuff_prompt_override = """Given this text extracts:
-----
{context}
-----
Please answer the following question:
{query}"""
prompt = PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Instantiate the chain
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=reordered_docs, query=query)

'\n\n科比打输了复活赛。'